In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [179]:
df=pd.read_csv("water_potability.csv")
# df1=df.copy()

In [180]:
df.drop(3014,inplace=True)

In [181]:
df_1 =df[df['Potability']==1]
df_0 = df[df['Potability']==0]
# no_water = df.dropna()
cols = list(df_1.columns)

In [182]:
def change(df_1,column, upper_limit,lis,lower_limit):
    for i in range(len(df_1)):
        x = df_1.iloc[i:i+1][column].values[0]
        if (x>=upper_limit):
            lis[i] = 0
        if(column=='ph'):
            if(x<=lower_limit):
                lis[i] = 0
    return lis

In [183]:
lis = df_1['Potability'].values
lis1 = df_0['Potability'].values
for i in cols[:-1]:
    Q1 = df_1[i].quantile(0.25)
    Q3 =df_1[i].quantile(0.75)
    iqr = Q3 - Q1
    upper_limit = Q3 + 1.5* iqr
    lower_limit = Q1 - 1.5 * iqr
    print(i,upper_limit,lower_limit)
    lis = change(df_1,i,upper_limit, lis, lower_limit)
# print(len(lis))

ph 10.563701112307495 3.5486786568579656
Hardness 283.5127538042832 108.82119802510387
Solids 46429.61356370653 -2787.3920824244906
Chloramines 11.356953261826861 2.9364417441007067
Sulfate 463.70770741945415 202.99741059989327
Conductivity 668.981243898836 176.11368975582258
Organic_carbon 22.83976790942959 5.550374644552143
Trihalomethanes 109.43106477190084 23.96415954481703
Turbidity 6.127557485528579 1.8129204506468555


In [184]:
dff=pd.DataFrame()   
for i in cols:
    dff[i] = df_1[i]
# df['Potability'] = lis 
# dff.shape

In [185]:
dff['Potability'] = lis

In [186]:
final_df = pd.concat([df_0, dff])
final_df1 = final_df[final_df['Potability']==1]
final_df2 = final_df[final_df['Potability']==0]

In [187]:
df1=dff.dropna()
df2=df1[df1['Potability']==1]

In [188]:
for i in cols:
    final_df1.fillna(df2[i].mean(),inplace=True)

C:\Users\mamid\anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [189]:
for i in cols:
    cx = df_0[i].max()
    print(cx)
    final_df2[i].fillna(float(cx), inplace = True)

13.999999999999998
304.23591211231667
61227.19600771213
12.653362021943142
460.1070689926536
753.3426195583046
28.30000000000001
120.03007700530677
6.739
0


C:\Users\mamid\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [190]:
f = pd.concat([final_df1, final_df2],ignore_index=True)
f

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,9.445130,145.805402,13168.529156,9.444471,310.583374,592.659021,8.606397,77.577460,3.875165,1
1,9.024845,128.096691,19859.676476,8.016423,300.150377,451.143481,14.770863,73.778026,3.985251,1
2,7.085586,169.974849,23403.637304,8.519730,7.085586,475.573562,12.924107,50.861913,2.747313,1
3,6.800119,242.008082,39143.403329,9.501695,187.170714,376.456593,11.432466,73.777275,3.854940,1
4,7.174135,203.408935,20401.102461,7.681806,287.085679,315.549900,14.533510,74.405616,3.939896,1
...,...,...,...,...,...,...,...,...,...,...
3270,4.767472,256.449917,48007.868134,5.773180,460.107069,590.429550,16.541561,44.938586,3.893378,0
3271,10.667364,173.381945,28912.202201,7.071294,276.634391,286.063394,17.685651,55.147364,4.135569,0
3272,10.808157,198.596751,29614.348790,5.782418,304.622061,383.269410,14.902820,47.896406,4.362542,0
3273,11.491011,94.812545,37188.826022,9.263166,258.930600,439.893618,16.172755,41.558501,4.369264,0


In [191]:
x=f.drop(['Potability'],axis=1)
y=f['Potability']

In [192]:
from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.25)


In [193]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25,random_state=88)
Xgb = XGBClassifier(max_depth=8, n_estimators =51,random_state=199)
Xgb.fit(x_train,y_train)
y_pred = Xgb.predict(x_test)
c = float(accuracy_score(y_test,y_pred))
print(c)

0.8315018315018315


In [194]:
import pickle
pickle_out=open("clsxgb.pkl","wb")
pickle.dump(Xgb,pickle_out)
pickle_out.close()

In [195]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25,random_state=49)
rfc = RandomForestClassifier(n_estimators= 117,
 max_features= 0.6, criterion="entropy",
 max_depth= 8,
 bootstrap= True,random_state=13);
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
accuracy_score(y_pred, y_test)


0.8412698412698413

In [196]:
import pickle
pickle_out=open("clsrf.pkl","wb")
pickle.dump(rfc,pickle_out)
pickle_out.close()

In [197]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=177)
log_clf = LogisticRegression(random_state=190,solver='saga',max_iter=100)
log_clf.fit(x_train,y_train)
y_predl=log_clf.predict(x_test)
accuracy_score(y_predl, y_test)


C:\Users\mamid\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.7572519083969466

In [198]:
import pickle
pickle_out=open("clslr.pkl","wb")
pickle.dump(log_clf,pickle_out)
pickle_out.close()

In [199]:
# op={}
# # from sklearn.linear_model import LogisticRegression
# for i in range(300,400):
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25,random_state=88)
#     Xgb = XGBClassifier(max_depth=8, n_estimators =51,random_state=i)
#     Xgb.fit(x_train,y_train)
#     y_pred = Xgb.predict(x_test)
# #     c = float(accuracy_score(y_test,y_pred))
#     a=accuracy_score(y_pred, y_test)
#     op.update({a:i})
# for k,v in op.items():    
#     if k==max(list(op.keys())):
#         print(k,v)

In [200]:
83 88
83 499
0.74 49
75 177

SyntaxError: invalid syntax (867294238.py, line 1)